In [140]:
import pandas as pd
import numpy as np
from sklearn.datasets import fetch_openml

In [137]:
adult = fetch_openml(name='adult', version=2)

In [127]:
for key, val in adult.items():
    print('-'*100)
    print(key)
    print('-'*100)
    print(val)
    print()

----------------------------------------------------------------------------------------------------
data
----------------------------------------------------------------------------------------------------
        age     workclass    fnlwgt     education  education-num  \
0      25.0       Private  226802.0          11th            7.0   
1      38.0       Private   89814.0       HS-grad            9.0   
2      28.0     Local-gov  336951.0    Assoc-acdm           12.0   
3      44.0       Private  160323.0  Some-college           10.0   
4      18.0           NaN  103497.0  Some-college           10.0   
...     ...           ...       ...           ...            ...   
48837  27.0       Private  257302.0    Assoc-acdm           12.0   
48838  40.0       Private  154374.0       HS-grad            9.0   
48839  58.0       Private  151910.0       HS-grad            9.0   
48840  22.0       Private  201490.0       HS-grad            9.0   
48841  52.0  Self-emp-inc  287927.0       HS-

In [128]:
print(adult['feature_names'])

['age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week', 'native-country']


In [129]:
adult['frame']['class'].value_counts()

<=50K    37155
>50K     11687
Name: class, dtype: int64

In [130]:
adult['frame']['sex'].value_counts()

Male      32650
Female    16192
Name: sex, dtype: int64

In [131]:
df = adult['frame']
attr = 'marital-status'
df_temp = []
df_temp2 = []
for attr_val in df[attr].unique():
    s = df[df[attr] == attr_val]['class'].value_counts(normalize=True).rename(attr_val)
    s.index = s.index.astype(str)
    cnt = len(df[df[attr] == attr_val])
    s['cnt'] = cnt
    df_temp.append(s)
df_temp = pd.concat(df_temp, axis=1).T
df_temp.sort_values(by='>50K')

,<=50K,>50K,cnt
Never-married,0.954520,0.045480,16117.0
Separated,0.935294,0.064706,1530.0
Widowed,0.915679,0.084321,1518.0
Married-spouse-absent,0.907643,0.092357,628.0
Divorced,0.898839,0.101161,6633.0
Married-AF-spouse,0.621622,0.378378,37.0
Married-civ-spouse,0.553867,0.446133,22379.0


In [148]:
def get_target_frac(df, attrs, target_attr, target_attr_val, col_name):
    '''
    Return a dataframe with index on attrs and column about the fraction of target_attr_val 
    appearing in the target_attr column grouped by the attrs.
    '''
    res = df[df[target_attr] == target_attr_val].groupby(attrs).size().to_frame(col_name) 
    res[col_name] /= df.groupby(attrs).size()
    return res
    
def what_if_removed(df, control_attr, dependent_attr, target_attr, target_attr_val, index=None):
    target_frac_concatted = []
    
    # First, get the target fraction for all the control_attr
    target_frac_origin = get_target_frac(df, control_attr, target_attr, target_attr_val, 'origin')
    target_frac_concatted.append(target_frac_origin)
    for dependent_attr_val in df[dependent_attr].unique():
        df_what_if_removed = df.query(f'`{dependent_attr}` != "{dependent_attr_val}"') 
        target_frac_dependent = get_target_frac(df_what_if_removed, control_attr, target_attr, target_attr_val, dependent_attr_val)
        target_frac_concatted.append(target_frac_dependent)
    
    target_frac_concatted = pd.concat(target_frac_concatted, axis=1)
    if index:
        target_frac_concatted = target_frac_concatted.reindex(index)
    return target_frac_concatted

In [149]:
print(adult['feature_names'])

['age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week', 'native-country']


In [158]:
control_attr = 'education'
index = ['Prof-school', 'Doctorate']
dependent_attr = 'marital-status'
target_attr = 'class'
target_attr_val = '>50K'
df = adult['frame'].copy()
df['age'] = pd.cut(df['age'], bins=np.linspace(0, 100, 11).astype(int)).astype('str')
what_if_removed(df, control_attr, dependent_attr, target_attr, target_attr_val, index)

,origin,Never-married,Married-civ-spouse,Widowed,Divorced,Separated,Married-spouse-absent,Married-AF-spouse
education,,,,,,,,
Prof-school,0.739808,0.801439,0.474790,0.742718,0.760526,0.740606,0.741858,0.739496
Doctorate,0.725589,0.780684,0.518325,0.728055,0.736059,0.730703,0.729776,0.725126


# Why Prof-school has higher high-income rate than Doctorate?

* Reason 1. marital-status = "Married-civ-spouse". It might be due to that a married prof-school earns more than a married doctorate. 

* Reason 2. occupation = "Prof-specialty". It might be due to that a prof-school tends to have a higher pay in a position of "Porf-specialty" than a doctorate.

* Reason 3. workclass = "Self-emp-not-inc" or "Self-emp-inc". It might be due to that a prof-school tends to earn more if s/he is self-employed.

* Reason 4. relationship = "Husband". It might be due to that a husband with prof-school degree tends to earn more than a huasband with doctorate degree.

* Reason 5. race = "White". It might be due that a white person with prof-school degree tends to earn more than a white person with doctorate degree. Or we say, if we don't take white people into account, prof-school degree tends to earn less than doctorate degree.

* Reason 6. sex = "Male".

* Reason 7. native_country = "United-States". 

* Reason 8. age = (30, 40] or (40, 50]. 

In [75]:
control_attr = 'workclass'
dependent_attr = 'marital-status'
target_attr = 'class'
target_attr_val = '>50K'
df = adult['frame']

pivot = (df[df[target_attr] == target_attr_val]
            .groupby([control_attr, dependent_attr]).size().to_frame(target_attr_val)
        )
pivot[target_attr_val] /= df.groupby([control_attr, dependent_attr]).size()#.to_frame('total')
pivot = pd.pivot_table(
    pivot.reset_index(),
    values=target_attr_val,
    index=control_attr,
    columns=dependent_attr,
    aggfunc='sum'
)
pivot

marital-status,Married-civ-spouse,Divorced,Never-married,Separated,Widowed,Married-spouse-absent,Married-AF-spouse
workclass,,,,,,,
Private,0.433981,0.091238,0.039860,0.064343,0.079596,0.079470,0.416667
Self-emp-not-inc,0.355129,0.155093,0.119086,0.035294,0.118110,0.208333,0.666667
Self-emp-inc,0.670886,0.260274,0.142180,0.240000,0.365854,0.125000,0.000000
Federal-gov,0.653259,0.151261,0.097826,0.179487,0.125000,0.200000,0.666667
Local-gov,0.511719,0.109641,0.077694,0.080000,0.071429,0.090909,0.000000
State-gov,0.506742,0.107595,0.051887,0.046154,0.106383,0.160000,0.000000
Without-pay,0.153846,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Never-worked,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [76]:
# https://smallbiztrends.com/2013/11/incorporated-versus-unincorporated-self-employment.html

In [154]:
df = adult['frame'].copy()
df['age'] = pd.cut(df['age'], bins=np.linspace(0, 100, 11).astype(int)).astype('str')

In [156]:
df.to_csv('adult.csv', index=False)

In [157]:
df

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,class
0,"(20, 30]",Private,226802.0,11th,7.0,Never-married,Machine-op-inspct,Own-child,Black,Male,0.0,0.0,40.0,United-States,<=50K
1,"(30, 40]",Private,89814.0,HS-grad,9.0,Married-civ-spouse,Farming-fishing,Husband,White,Male,0.0,0.0,50.0,United-States,<=50K
2,"(20, 30]",Local-gov,336951.0,Assoc-acdm,12.0,Married-civ-spouse,Protective-serv,Husband,White,Male,0.0,0.0,40.0,United-States,>50K
3,"(40, 50]",Private,160323.0,Some-college,10.0,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688.0,0.0,40.0,United-States,>50K
4,"(10, 20]",NaN,103497.0,Some-college,10.0,Never-married,NaN,Own-child,White,Female,0.0,0.0,30.0,United-States,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48837,"(20, 30]",Private,257302.0,Assoc-acdm,12.0,Married-civ-spouse,Tech-support,Wife,White,Female,0.0,0.0,38.0,United-States,<=50K
48838,"(30, 40]",Private,154374.0,HS-grad,9.0,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0.0,0.0,40.0,United-States,>50K
48839,"(50, 60]",Private,151910.0,HS-grad,9.0,Widowed,Adm-clerical,Unmarried,White,Female,0.0,0.0,40.0,United-States,<=50K
48840,"(20, 30]",Private,201490.0,HS-grad,9.0,Never-married,Adm-clerical,Own-child,White,Male,0.0,0.0,20.0,United-States,<=50K
